In [0]:
!kill -9 -1

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPool1D
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os

from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.layers import BatchNormalization, SpatialDropout1D, LSTM, TimeDistributed



from keras.callbacks import Callback
from keras.preprocessing import text


from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os
import nltk
from nltk import tokenize 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv') 

In [83]:
train['mal']         = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)

test_labelled.comment_text.fillna("empty", inplace=True)
test_unlabelled.comment_text.fillna("empty", inplace=True) 
train.comment_text = train.comment_text.replace(np.nan, '', regex=True)

# ADDITIONAL PREPROCESSING
def clean_str(string):
    string = string.replace(".....", ".").replace("....", ".").replace("...", ".").replace("..", ".")
    string = string.replace("!!!!!", "!").replace("!!!!", "!").replace("!!!", "!").replace("!!", "!")
    string = string.replace("?????", "?").replace("????", "?").replace("???", "?").replace("??", "?")
    string = string.replace(".jpg", " jpg")
    string = string.replace(" ?", "?")
    string = string.replace(" .", ".")
    string = string.replace(" !", "!")
    return string.strip().lower()
  

train.comment_text_new          = train.comment_text.apply(lambda x: clean_str(x)) 
test_labelled.comment_text_new  = test_labelled.comment_text.apply(lambda x: clean_str(x)) 


# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

X      = pd.Series(X)
X_test = pd.Series(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [0]:
max_features = 400000
maxlen       = 400
dropout_rate = 0.5
rs           = 422
epochs       = 4
batch_size   = 32
embed_dim    = 150
rec_units    = 50


seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [0]:
def HAN(max_features, dropout_rate, embed_dim, rec_units, max_sen_len, max_sent, reduction = 'average'):
    if K.backend == 'tensorflow':        
        K.clear_session()
    sentence_input = Input(shape=(max_sen_len,), dtype='int32')
    embedded_sequences = Embedding(max_features+1, embed_dim, trainable=True)(sentence_input)
    embedded_sequences = SpatialDropout1D(dropout_rate)(embedded_sequences)
    l_lstm = Bidirectional(CuDNNGRU(rec_units, return_sequences=True, kernel_regularizer=l2(0.1)))(embedded_sequences)
    l_att = AttentionWithContext()(l_lstm)
    sentEncoder = Model(sentence_input, l_att)

    comment_input = Input(shape=(max_sent, max_sen_len), dtype='int32')
    comment_encoder = TimeDistributed(sentEncoder)(comment_input)
    l_lstm_sent = Bidirectional(CuDNNGRU(rec_units, return_sequences=True,  kernel_regularizer=l2(0.1)))(comment_encoder)
    l_att_sent = AttentionWithContext()(l_lstm_sent)

    preds = Dense(1, activation='sigmoid')(l_att_sent)
    model = Model(comment_input, preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=10, clipnorm=10),
                  metrics=['acc'])
    return model
    #print( model.summary())
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0
tokenizer = text.Tokenizer(num_words=max_features, oov_token='oov')
tokenizer.fit_on_texts(X)

In [0]:
def sent_tok(text, max_sent=15, max_sen_len=25, na_element='empty'):
  sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
  import pdb
  #pdb.set_trace()
  #sent_text = sequence.pad_sequences(sent_text, maxlen=max_sent)
  N = max_sent - len(sent_text)
  N = max(0, N)
  sent_text = (sent_text + N * [na_element])[:max_sent]
  pdb.set_trace()
  #sent_text = sent_text[:max_sent]
  # now loop over each sentence and tokenize it separately
  for i, sentence in enumerate(sent_text):
    N_s = max_sen_len - len(sentence.split())
    N_s = max(0, N_s)
    sent_text[i] = (sentence.split() + N_s * [na_element])[:max_sen_len]
    #sent_text[i] = ' '.join(sent_text[i])
    
    sent_text[i] = tokenizer.texts_to_sequences(sent_text[i])
  #    tokenized_text = tokenizer.texts_to_sequences(sentence)
  #sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
  return sent_text

In [90]:
def clean_str(string):
    string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    #temp = t.texts_to_sequences(s.replace("?", " ").replace("!", " ").replace(".", " ").split())
    temp = tokenizer.texts_to_sequences(s)
    #print(s)
    if len(temp)==0:
        return [[0]]
    #else:
    #    temp = np.concatenate(temp).astype(int)
    #if type(temp[0])==list:
    #    temp = [item for sublist in temp for item in sublist]
    return temp#.reshape(-1)  

padding = True
max_sent = 5
max_sen_len = 100
train_posts = []
train_labels = []
train_texts = []
import re
from keras.preprocessing.sequence import pad_sequences
# FULL TRAIN
for i, value in enumerate(X):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    #train_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = [re.sub(r"[^A-Za-z0-9]", " ", i).strip() for i in sentences]
    sentences = [s for s in sentences if len(s.split())>1]
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent
    if len(sentences)>max_sent:
        sentences = sentences[0:max_sent]
    sentences = pad_sequences(sentences, maxlen=max_sen_len, padding='post')
     
    train_posts.append(sentences)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000


In [0]:
X = np.array(train_posts)

In [0]:
print('Preparing train set')
#X         = X[:1000].apply(lambda x: sent_tok(x, 15, 25))

print('Preparing test set')
#X_test    = X_test[:1000].apply(lambda x: sent_tok(x, 15, 25))

Preparing train set
Preparing test set


### HAN

In [0]:
np.unique(y)

array([False,  True])

In [0]:
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
   
    #tokenizer = text.Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(pd.concat([X_train, unlab.comment_text], axis=0))
    #X_val   = X_val.apply(lambda x: sent_tok(x, 15, 25))
    #X_train = X_train.apply(lambda x: sent_tok(x, 15, 25))
    
    #X_train = X_train.values
    #X_val   = X_val.values
   
    model                = HAN(max_features, dropout_rate, embed_dim, rec_units, 15, 5)
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=False, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    
    model.save_weights(f'{cv_models_path}/HAN_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    del model

 fold 0
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 63s 499us/step - loss: 54.1425 - acc: 0.2637 - val_loss: 53.8673 - val_acc: 0.4547
Epoch 2/4
126974/126974 [==============================] - 52s 410us/step - loss: 53.5948 - acc: 0.7651 - val_loss: 53.3217 - val_acc: 0.8995
Epoch 3/4
 65280/126974 [==============>...............] - ETA: 24s - loss: 53.1827 - acc: 0.8987

KeyboardInterrupt: ignored

In [0]:
data = pd.DataFrame({'acc':history.history['acc'],
                    'loss': history.history['loss'],
                    'val_acc': history.history['val_acc'],
                    'val_loss': history.history['val_loss']})

In [0]:
np.array(auc).mean()

In [0]:
np.array(roc).mean()

In [73]:
X_train

array([[[   56,     7,    74, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]],

       [[  153,     2,    14, ...,     0,     0,     0],
        [  527,     2,    83, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]],

       [[ 1642,     9,    14, ...,     0,     0,     0],
        [ 1199,     2,  7985, ...,     0,     0,     0],
        [   13,    44,    55, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,    

In [0]:
y_train = y*1000

In [97]:
X_train   = X
#y_train   = y
from keras.preprocessing import text
#tokenizer = text.Tokenizer(num_words=max_features, oov_token='unknown')
#tokenizer.fit_on_texts(X_train)

#list_tokenized_train = tokenizer.texts_to_sequences(X_train)
#list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
#X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_test  = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

model   = HAN(max_features, dropout_rate, embed_dim, rec_units, max_sen_len, max_sent)

y_train = np.array(y_train)
y_test  = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train,   batch_size=256, epochs=epochs, shuffle=False, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/HAN.h5')
del model

Fitting
Epoch 1/4
158719/158719 [==============================] - 75s 474us/step - loss: -1553.4322 - acc: 0.0015
Epoch 2/4
158719/158719 [==============================] - 71s 445us/step - loss: -1584.9086 - acc: 0.0000e+00
Epoch 3/4
158719/158719 [==============================] - 71s 445us/step - loss: -1586.3397 - acc: 0.0000e+00
Epoch 4/4
158719/158719 [==============================] - 71s 446us/step - loss: -1586.3397 - acc: 0.0000e+00


ValueError: ignored

In [0]:
auc_f

In [0]:
roc_f

### Maximum Pooling

In [0]:
c=0 # added later, this is why model shows folds starting with 5
# MAXIMUM POOLING
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    #tokenizer = text.Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(pd.concat([X_train, unlab.comment_text], axis=0))
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction='maximum')
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=False, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/BGRU_maxpool_fold_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    del model

 fold 5
Fitting
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 63s 493us/step - loss: 0.1689 - acc: 0.9378 - val_loss: 0.1547 - val_acc: 0.9455
Epoch 2/4
126974/126974 [==============================] - 61s 480us/step - loss: 0.1086 - acc: 0.9578 - val_loss: 0.1495 - val_acc: 0.9476
Epoch 3/4
126974/126974 [==============================] - 61s 480us/step - loss: 0.1010 - acc: 0.9604 - val_loss: 0.1466 - val_acc: 0.9502
Epoch 4/4
31745/31745 [==============================] - 5s 160us/step
 average precision 0.8772565344100596
 roc auc 0.9720347282415762
 fold 6
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 63s 494us/step - loss: 0.1720 - acc: 0.9386 - val_loss: 0.1506 - val_acc: 0.9471
Epoch 2/4
126974/126974 [==============================] - 61s 478us/step - loss: 0.1084 - 

In [0]:
np.array(auc).mean()

0.8521627725963542

In [0]:
np.array(roc).mean()

0.9689112874345079

In [0]:
X_train = X
y_train = y

tokenizer            = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
#list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

model = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction='maximum')

y_train              = np.array(y_train)
y_test               = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/BGRU_maxpool.h5')
del model

Fitting
Epoch 1/4
158719/158719 [==============================] - 73s 458us/step - loss: 0.1606 - acc: 0.9419
Epoch 2/4
158719/158719 [==============================] - 71s 446us/step - loss: 0.1049 - acc: 0.9596
Epoch 3/4
158719/158719 [==============================] - 71s 446us/step - loss: 0.0986 - acc: 0.9620
Epoch 4/4
64830/64830 [==============================] - 10s 157us/step


In [0]:
auc_f

0.853890338923986

In [0]:
roc_f

0.9714551605769767

### ATTENTION

In [0]:
c = 0
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    #tokenizer = text.Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(pd.concat([X_train, unlab.comment_text], axis=0))
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction='attention')
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=False, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/BGRU_attention_fold_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    c += 1
    del model

 fold 0
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 79s 625us/step - loss: 0.2265 - acc: 0.9216 - val_loss: 0.1544 - val_acc: 0.9446
Epoch 2/4
126974/126974 [==============================] - 75s 594us/step - loss: 0.1585 - acc: 0.9434 - val_loss: 0.1761 - val_acc: 0.9452
Epoch 3/4
126974/126974 [==============================] - 75s 592us/step - loss: 0.1132 - acc: 0.9574 - val_loss: 0.1560 - val_acc: 0.9476
Epoch 4/4
31745/31745 [==============================] - 8s 238us/step
 average precision 0.869315370688949
 roc auc 0.9673301881493371
 fold 1
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 79s 625us/step - loss: 0.2154 - acc: 0.9248 - val_loss: 0.1573 - val_acc: 0.9432
Epoch 2/4
126974/126974 [==============================] - 75s 593us/step - loss: 0.1260 - acc: 0.9536 - val_loss: 0.1561 - val_acc: 0.9458
Epoch 3/4
126974/126974 [=====

In [0]:
np.array(auc).mean()

0.8525556411291078

In [0]:
np.array(roc).mean()

0.9667035973653868

In [0]:
X_train = X
y_train = y

tokenizer            = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
#list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

model = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units, reduction='attention')

y_train              = np.array(y_train)
y_test               = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train, batch_size=batch_size, epochs=4, shuffle=False, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)

model.save_weights(f'{models_path}/BGRU_attention.h5')

auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
del model

Fitting
Epoch 1/4
158719/158719 [==============================] - 90s 567us/step - loss: 0.2186 - acc: 0.9226
Epoch 2/4
158719/158719 [==============================] - 85s 537us/step - loss: 0.1171 - acc: 0.9563
Epoch 3/4
158719/158719 [==============================] - 85s 538us/step - loss: 0.1066 - acc: 0.9597
Epoch 4/4
64830/64830 [==============================] - 15s 227us/step


In [0]:
auc_f

0.8489225428060403

In [0]:
roc_f

0.9676813223421821